In [0]:
# %sql
# select * from foi_mod.foirawrequests
# order by requestid desc
# limit 200;

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, LongType, DoubleType,
    BooleanType, TimestampType, DateType, ArrayType, MapType
)
from delta.tables import DeltaTable # For merge operations
from pyspark.sql.functions import current_timestamp, col
from datetime import datetime

import sys
repo_root_path = "/Workspace/Users/richard.qi@gov.bc.ca/foi-reporting/databricks"
# Add the repository root to sys.path if it's not already there
if repo_root_path not in sys.path:
    sys.path.insert(0, repo_root_path) # Insert at the beginning to prioritize

from Shared.job_tracker import (
    get_last_successful_run_time,
    record_job_status,
    get_current_run_id,
    generate_date_range_json
)

dbutils.widgets.text("bucket", "DataStagingBucket", "S3 Bucket")
dbutils.widgets.text("env", "prod", "Environment")

# -------------------- Configuration ----------------------
JOB_NAME = "Daily_FOIMOD_DB_Loading"
JOB_TRACKER_TABLE_PATH = "/mnt/delta/tables/job_tracker"

# Initialize Spark Session
spark = SparkSession.builder.appName(f"{JOB_NAME}_APP").getOrCreate()

# Get the current Databricks run ID (or a generated UUID)
current_run_id = get_current_run_id(spark)
print(f"Starting job: {JOB_NAME} with run_id: {current_run_id}")
job_start_timestamp = datetime.now()
status_message = None

# --- S3 Configuration ---
# s3_bucket = spark.conf.get("spark.s3_bucket")
s3_bucket = dbutils.widgets.get("bucket")
env = dbutils.widgets.get("env")
S3_BASE_PATH = f"s3a://{s3_bucket}/daily_exports/{env}/foidb/"
# now = datetime.now()
# formatted_date = now.strftime("%Y-%m-%d")
# S3_BASE_PATH = f"s3a://{s3_bucket}/daily_exports/dev/foidb/{formatted_date}/"
# S3_BASE_PATH = f"s3a://{s3_bucket}/daily_exports/dev/foidb/2025-06-03/"

# Base path for your Delta tables
DELTA_BASE_PATH = "dbfs:/user/hive/warehouse/foi_mod.db"

# Array of S3 JSON file paths.
json_files_to_process = [
    "ApplicantCategories.json",
    "ApplicantCorrespondenceTags.json",
    "ApplicantCorrespondenceTemplates.json",
    "AXISMigrationMapper.json",
    "CFRFEEStatuses.json",
    "CFRFormReasons.json",
    "CloseReasons.json",
    "CommentTypes.json",
    "ContactTypes.json",
    "DeliveryModes.json",
    "DocumentPathMapper.json",
    "DocumentTemplates.json",
    "DocumentTypes.json",
    "ExtensionReasons.json",
    "ExtensionStatuses.json",
    "FeeCodes.json",
    "FOIApplicantCorrespondenceAttachments.json",
    "FOIApplicantCorrespondenceAttachmentsRawRequests.json",
    "FOIApplicantCorrespondenceEmails.json",
    "FOIApplicantCorrespondenceEmailsRawRequests.json",
    "FOIApplicantCorrespondenceResponses.json",
    "FOIApplicantCorrespondences.json",
    "FOIApplicantCorrespondencesRawRequests.json",
    "FOIApplicantCorrespondencesTags.json",
    "FOIAssignees.json",
    "FOICorrespondenceEmails.json",
    "FOICorrespondenceEmailsRawRequests.json",
    "FOIMinistryRequestDivisions.json",
    "FOIMinistryRequestDocuments.json",
    "FOIMinistryRequests.json",
    "FOIMinistryRequestSubjectCodes.json",
    "FOIRawRequestComments.json",
    "FOIRawRequestDocuments.json",
    "FOIRawRequestNotifications.json",
    "FOIRawRequestNotificationUsers.json",
    "FOIRawRequests.json",
    "FOIRawRequestWatchers.json",
    "FOIRequestApplicantMappings.json",
    "FOIRequestApplicants.json",
    "FOIRequestApplicantionFeeReceipts.json",
    "FOIRequestApplicationFees.json",
    "FOIRequestCFRFees.json",
    "FOIRequestComments.json",
    "FOIRequestContactInformation.json",
    "FOIRequestExtensionDocumentMapping.json",
    "FOIRequestExtensions.json",
    "FOIRequestNotifications.json",
    "FOIRequestNotificationUsers.json",
    "FOIRequestOIPC.json",
    "FOIRequestPaymens.json",
    "FOIRequestPersonalAttributes.json",
    "FOIRequestRecords.json",
    "FOIRequests.json",
    "FOIRequestStatuses.json",
    "FOIRequestTeams.json",
    "FOIRequestWatchers.json",
    "FOIRestrictedMinistryRequests.json",
    "FOIUsers.json",
    "HistoricalRecords.json",
    "NotificationTypes.json",
    "NotificationUserTypes.json",
    "OIPCInquiryoutcomes.json",
    "OIPCOutcomes.json",
    "OIPCReasons.json",
    "OIPCReviewTypes.json",
    "OIPCReviewTypesReasons.json",
    "OIPCStatuses.json",
    "OperatingTeamEmails.json",
    "OperatingTeams.json",
    "Payments.json",
    "PersonalInformationAttributes.json",
    "ProgramAreaDivisions.json",
    "ProgramAreaDivisionStages.json",
    "ProgramAreas.json",
    "ReceivedModes.json",
    "RequestorTypes.json",
    "RevenueAccounts.json",
    "SubjectCodes.json",
    "UnopenedReport.json",
    "FOIOpenInformationRequests.json",
    "OpenInfoPublicationStatuses.json",
    "OpenInformationExemptions.json",
    "OpenInformationStatuses.json"
]

# --- table_mapping ---
table_mappings = {
    "applicantcategories": {
        "source_table_schema": StructType([
            StructField("applicantcategoryid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "applicantcategories",
        "primary_keys": ["applicantcategoryid"],
        "transform_map": {
            "applicantcategoryid": ("applicantcategoryid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "applicantcorrespondencetags": {
        "source_table_schema": StructType([
            StructField("tagid", StringType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "applicantcorrespondencetags",
        "primary_keys": ["tagid"],
        "transform_map": {
            "tagid": ("tagid", StringType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "applicantcorrespondencetemplates": {
        "source_table_schema": StructType([
            StructField("templateid", IntegerType(), True),
            StructField("documenturipath", StringType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("active", StringType(), True),
            StructField("version", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("display", StringType(), True)
        ]),
        "target_table_name": "applicantcorrespondencetemplates",
        "primary_keys": ["templateid"],
        "transform_map": {
            "templateid": ("templateid", IntegerType()),
            "documenturipath": ("documenturipath", StringType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "active": ("active", StringType()),
            "version": ("version", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "display": ("display", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "axismigrationmapper": {
        "source_table_schema": StructType([
            StructField("stateonAXIS", StringType(), True),
            StructField("stateonFOI", StringType(), True)
        ]),
        "target_table_name": "axismigrationmapper",
        "primary_keys": ["stateonAXIS", "stateonFOI"],
        "transform_map": {
            "stateonAXIS": ("stateonAXIS", StringType()),
            "stateonFOI": ("stateonFOI", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "cfrfeestatuses": {
        "source_table_schema": StructType([
            StructField("cfrfeestatusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "cfrfeestatuses",
        "primary_keys": ["cfrfeestatusid"],
        "transform_map": {
            "cfrfeestatusid": ("cfrfeestatusid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "cfrformreasons": {
        "source_table_schema": StructType([
            StructField("cfrformreasonid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "cfrformreasons",
        "primary_keys": ["cfrformreasonid"],
        "transform_map": {
            "cfrformreasonid": ("cfrformreasonid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "closereasons": {
        "source_table_schema": StructType([
            StructField("closereasonid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True)
        ]),
        "target_table_name": "closereasons",
        "primary_keys": ["closereasonid"],
        "transform_map": {
            "closereasonid": ("closereasonid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "commenttypes": {
        "source_table_schema": StructType([
            StructField("commenttypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("label", StringType(), True)
        ]),
        "target_table_name": "commenttypes",
        "primary_keys": ["commenttypeid"],
        "transform_map": {
            "commenttypeid": ("commenttypeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "label": ("label", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "contacttypes": {
        "source_table_schema": StructType([
            StructField("contacttypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "contacttypes",
        "primary_keys": ["contacttypeid"],
        "transform_map": {
            "contacttypeid": ("contacttypeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "deliverymodes": {
        "source_table_schema": StructType([
            StructField("deliverymodeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "deliverymodes",
        "primary_keys": ["deliverymodeid"],
        "transform_map": {
            "deliverymodeid": ("deliverymodeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "documentpathmapper": {
        "source_table_schema": StructType([
            StructField("documentpathid", IntegerType(), True),
            StructField("category", StringType(), True),
            StructField("bucket", StringType(), True),
            StructField("attributes", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("updated_at", TimestampType(), True)
        ]),
        "target_table_name": "documentpathmapper",
        "primary_keys": ["documentpathid"],
        "transform_map": {
            "documentpathid": ("documentpathid", IntegerType()),
            "category": ("category", StringType()),
            "bucket": ("bucket", StringType()),
            "attributes": ("attributes", StringType()),
            "isactive": ("isactive", StringType()),
            "createdby": ("createdby", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "documenttemplates": {
        "source_table_schema": StructType([
            StructField("template_id", IntegerType(), True),
            StructField("cdogs_hash_code", StringType(), True),
            StructField("extension", StringType(), True),
            StructField("document_type_id", IntegerType(), True)
        ]),
        "target_table_name": "documenttemplates",
        "primary_keys": ["template_id"],
        "transform_map": {
            "template_id": ("template_id", IntegerType()),
            "cdogs_hash_code": ("cdogs_hash_code", StringType()),
            "extension": ("extension", StringType()),
            "document_type_id": ("document_type_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "documenttypes": {
        "source_table_schema": StructType([
            StructField("document_type_id", IntegerType(), True),
            StructField("document_type_name", StringType(), True),
            StructField("description", StringType(), True)
        ]),
        "target_table_name": "documenttypes",
        "primary_keys": ["document_type_id"],
        "transform_map": {
            "document_type_id": ("document_type_id", IntegerType()),
            "document_type_name": ("document_type_name", StringType()),
            "description": ("description", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "extensionreasons": {
        "source_table_schema": StructType([
            StructField("extensionreasonid", IntegerType(), True),
            StructField("reason", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("extensiontype", StringType(), True),
            StructField("defaultextendedduedays", IntegerType(), True)
        ]),
        "target_table_name": "extensionreasons",
        "primary_keys": ["extensionreasonid"],
        "transform_map": {
            "extensionreasonid": ("extensionreasonid", IntegerType()),
            "reason": ("reason", StringType()),
            "isactive": ("isactive", StringType()),
            "extensiontype": ("extensiontype", StringType()),
            "defaultextendedduedays": ("defaultextendedduedays", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "extensionstatuses": {
        "source_table_schema": StructType([
            StructField("extensionstatusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "extensionstatuses",
        "primary_keys": ["extensionstatusid"],
        "transform_map": {
            "extensionstatusid": ("extensionstatusid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "feecodes": {
        "source_table_schema": StructType([
            StructField("fee_code_id", IntegerType(), True),
            StructField("code", StringType(), True),
            StructField("description", StringType(), True),
            StructField("start_date", DateType(), True),
            StructField("end_date", DateType(), True),
            StructField("fee", IntegerType(), True),
            StructField("revenue_account_id", IntegerType(), True)
        ]),
        "target_table_name": "feecodes",
        "primary_keys": ["fee_code_id"],
        "transform_map": {
            "fee_code_id": ("fee_code_id", IntegerType()),
            "code": ("code", StringType()),
            "description": ("description", StringType()),
            "start_date": ("start_date", DateType()),
            "end_date": ("end_date", DateType()),
            "fee": ("fee", IntegerType()),
            "revenue_account_id": ("revenue_account_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondenceattachments": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceattachmentid", IntegerType(), True),
            StructField("applicantcorrespondenceid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", StringType(), True),
            StructField("attachmentfilename", StringType(), True),
            StructField("attachmentdocumenturipath", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("applicantcorrespondence_version", IntegerType(), True),
            StructField("version", IntegerType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondenceattachments",
        "primary_keys": ["applicantcorrespondenceattachmentid", "version"],
        "transform_map": {
            "applicantcorrespondenceattachmentid": ("applicantcorrespondenceattachmentid", IntegerType()),
            "applicantcorrespondenceid": ("applicantcorrespondenceid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", StringType()),
            "attachmentfilename": ("attachmentfilename", StringType()),
            "attachmentdocumenturipath": ("attachmentdocumenturipath", StringType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "applicantcorrespondence_version": ("applicantcorrespondence_version", IntegerType()),
            "version": ("version", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondenceattachmentsrawrequests": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceattachmentid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("applicantcorrespondenceid", IntegerType(), True),
            StructField("applicantcorrespondence_version", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("attachmentfilename", StringType(), True),
            StructField("attachmentdocumenturipath", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondenceattachmentsrawrequests",
        "primary_keys": ["applicantcorrespondenceattachmentid", "version"],
        "transform_map": {
            "applicantcorrespondenceattachmentid": ("applicantcorrespondenceattachmentid", IntegerType()),
            "version": ("version", IntegerType()),
            "applicantcorrespondenceid": ("applicantcorrespondenceid", IntegerType()),
            "applicantcorrespondence_version": ("applicantcorrespondence_version", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "attachmentfilename": ("attachmentfilename", StringType()),
            "attachmentdocumenturipath": ("attachmentdocumenturipath", StringType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondenceemails": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceemailid", IntegerType(), True),
            StructField("applicantcorrespondence_id", IntegerType(), True),
            StructField("applicantcorrespondence_version", IntegerType(), True),
            StructField("correspondence_to", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondenceemails",
        "primary_keys": ["applicantcorrespondenceemailid"],
        "transform_map": {
            "applicantcorrespondenceemailid": ("applicantcorrespondenceemailid", IntegerType()),
            "applicantcorrespondence_id": ("applicantcorrespondence_id", IntegerType()),
            "applicantcorrespondence_version": ("applicantcorrespondence_version", IntegerType()),
            "correspondence_to": ("correspondence_to", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondenceemailsrawrequests": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceemailid", IntegerType(), True),
            StructField("applicantcorrespondence_id", IntegerType(), True),
            StructField("applicantcorrespondence_version", IntegerType(), True),
            StructField("correspondence_to", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondenceemailsrawrequests",
        "primary_keys": ["applicantcorrespondenceemailid"],
        "transform_map": {
            "applicantcorrespondenceemailid": ("applicantcorrespondenceemailid", IntegerType()),
            "applicantcorrespondence_id": ("applicantcorrespondence_id", IntegerType()),
            "applicantcorrespondence_version": ("applicantcorrespondence_version", IntegerType()),
            "correspondence_to": ("correspondence_to", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondenceresponses": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceresponseid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("response_at", TimestampType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("applicantcorrespondence_id", IntegerType(), True),
            StructField("applicantcorrespondenceversion_id", IntegerType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondenceresponses",
        "primary_keys": ["applicantcorrespondenceresponseid", "version"],
        "transform_map": {
            "applicantcorrespondenceresponseid": ("applicantcorrespondenceresponseid", IntegerType()),
            "version": ("version", IntegerType()),
            "response_at": ("response_at", TimestampType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "applicantcorrespondence_id": ("applicantcorrespondence_id", IntegerType()),
            "applicantcorrespondenceversion_id": ("applicantcorrespondenceversion_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondences": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceid", IntegerType(), True),
            StructField("parentapplicantcorrespondenceid", IntegerType(), True),
            StructField("templateid", IntegerType(), True),
            StructField("correspondencemessagejson", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("sentcorrespondencemessage", StringType(), True),
            StructField("sent_at", TimestampType(), True),
            StructField("version", IntegerType(), True),
            StructField("isdraft", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("isresponse", StringType(), True),
            StructField("response_at", TimestampType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondences",
        "primary_keys": ["applicantcorrespondenceid", "version"],
        "transform_map": {
            "applicantcorrespondenceid": ("applicantcorrespondenceid", IntegerType()),
            "parentapplicantcorrespondenceid": ("parentapplicantcorrespondenceid", IntegerType()),
            "templateid": ("templateid", IntegerType()),
            # "correspondencemessagejson": ("correspondencemessagejson", StringType()),         #skip correspondencemessagejson, it won't be used in ETL and data warehouse
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "sentcorrespondencemessage": ("sentcorrespondencemessage", StringType()),
            "sent_at": ("sent_at", TimestampType()),
            "version": ("version", IntegerType()),
            "isdraft": ("isdraft", StringType()),
            "isdeleted": ("isdeleted", StringType()),
            "isresponse": ("isresponse", StringType()),
            "response_at": ("response_at", TimestampType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondencesrawrequests": {
        "source_table_schema": StructType([
            StructField("applicantcorrespondenceid", IntegerType(), True),
            StructField("parentapplicantcorrespondenceid", IntegerType(), True),
            StructField("templateid", IntegerType(), True),
            # StructField("correspondencemessagejson", StringType(), True),         #skip correspondencemessagejson, it won't be used in ETL and data warehouse
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("response_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foirawrequest_id", IntegerType(), True),
            StructField("foirawrequestversion_id", IntegerType(), True),
            StructField("sentcorrespondencemessage", StringType(), True),
            StructField("sent_at", TimestampType(), True),
            StructField("sentby", StringType(), True),
            StructField("version", IntegerType(), True),
            StructField("isdraft", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("isresponse", StringType(), True),
            StructField("israwrequest", StringType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondencesrawrequests",
        "primary_keys": ["applicantcorrespondenceid", "version"],
        "transform_map": {
            "applicantcorrespondenceid": ("applicantcorrespondenceid", IntegerType()),
            "parentapplicantcorrespondenceid": ("parentapplicantcorrespondenceid", IntegerType()),
            "templateid": ("templateid", IntegerType()),
            "correspondencemessagejson": ("correspondencemessagejson", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "response_at": ("response_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "foirawrequest_id": ("foirawrequest_id", IntegerType()),
            "foirawrequestversion_id": ("foirawrequestversion_id", IntegerType()),
            "sentcorrespondencemessage": ("sentcorrespondencemessage", StringType()),
            "sent_at": ("sent_at", TimestampType()),
            "sentby": ("sentby", StringType()),
            "version": ("version", IntegerType()),
            "isdraft": ("isdraft", StringType()),
            "isdeleted": ("isdeleted", StringType()),
            "isresponse": ("isresponse", StringType()),
            "israwrequest": ("israwrequest", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiapplicantcorrespondencetags": {
        "source_table_schema": StructType([
            StructField("tagid", IntegerType(), True),
            StructField("applicantcorrespondenceid", IntegerType(), True)
        ]),
        "target_table_name": "foiapplicantcorrespondencetags",
        "primary_keys": ["tagid", "applicantcorrespondenceid"],
        "transform_map": {
            "tagid": ("tagid", IntegerType()),
            "applicantcorrespondenceid": ("applicantcorrespondenceid", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiassignees": {
        "source_table_schema": StructType([
            StructField("foiassigneeid", IntegerType(), True),
            StructField("username", StringType(), True),
            StructField("firstname", StringType(), True),
            StructField("middlename", StringType(), True),
            StructField("lastname", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "foiassignees",
        "primary_keys": ["foiassigneeid"],
        "transform_map": {
            "foiassigneeid": ("foiassigneeid", IntegerType()),
            "username": ("username", StringType()),
            "firstname": ("firstname", StringType()),
            "middlename": ("middlename", StringType()),
            "lastname": ("lastname", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foicorrespondenceemails": {
        "source_table_schema": StructType([
            StructField("correspondenceemailid", IntegerType(), True),
            StructField("email", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True)
        ]),
        "target_table_name": "foicorrespondenceemails",
        "primary_keys": ["correspondenceemailid"],
        "transform_map": {
            "correspondenceemailid": ("correspondenceemailid", IntegerType()),
            "email": ("email", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foicorrespondenceemailsrawrequests": {
        "source_table_schema": StructType([
            StructField("correspondenceemailid", IntegerType(), True),
            StructField("email", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foirawrequest_id", IntegerType(), True),
            StructField("foirawrequestversion_id", IntegerType(), True)
        ]),
        "target_table_name": "foicorrespondenceemailsrawrequests",
        "primary_keys": ["correspondenceemailid"],
        "transform_map": {
            "correspondenceemailid": ("correspondenceemailid", IntegerType()),
            "email": ("email", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "foirawrequest_id": ("foirawrequest_id", IntegerType()),
            "foirawrequestversion_id": ("foirawrequestversion_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiministryrequestdivisions": {
        "source_table_schema": StructType([
            StructField("foiministrydivisionid", IntegerType(), True),
            StructField("divisionid", IntegerType(), True),
            StructField("stageid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("divisionduedate", TimestampType(), True),
            StructField("eapproval", StringType(), True),
            StructField("divisionreceiveddate", TimestampType(), True)
        ]),
        "target_table_name": "foiministryrequestdivisions",
        "primary_keys": ["foiministrydivisionid"],
        "transform_map": {
            "foiministrydivisionid": ("foiministrydivisionid", IntegerType()),
            "divisionid": ("divisionid", IntegerType()),
            "stageid": ("stageid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "divisionduedate": ("divisionduedate", TimestampType()),
            "eapproval": ("eapproval", StringType()),
            "divisionreceiveddate": ("divisionreceiveddate", TimestampType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiministryrequestdocuments": {
        "source_table_schema": StructType([
            StructField("foiministrydocumentid", IntegerType(), True),
            StructField("documentpath", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("filename", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("category", StringType(), True),
            StructField("version", IntegerType(), True)
        ]),
        "target_table_name": "foiministryrequestdocuments",
        "primary_keys": ["foiministrydocumentid", "version"],
        "transform_map": {
            "foiministrydocumentid": ("foiministrydocumentid", IntegerType()),
            "documentpath": ("documentpath", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "filename": ("filename", StringType()),
            "isactive": ("isactive", StringType()),
            "category": ("category", StringType()),
            "version": ("version", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiministryrequests": {
        "source_table_schema": StructType([
            StructField("foiministryrequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("isactive", StringType(), True),
            StructField("filenumber", StringType(), True),
            StructField("description", StringType(), True),
            StructField("recordsearchfromdate", TimestampType(), True),
            StructField("recordsearchtodate", TimestampType(), True),
            StructField("startdate", TimestampType(), True),
            StructField("duedate", TimestampType(), True),
            StructField("assignedto", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("programareaid", IntegerType(), True),
            StructField("requeststatusid", IntegerType(), True),
            StructField("foirequest_id", IntegerType(), True),
            StructField("foirequestversion_id", IntegerType(), True),
            StructField("assignedgroup", StringType(), True),
            StructField("assignedministryperson", StringType(), True),
            StructField("assignedministrygroup", StringType(), True),
            StructField("cfrduedate", TimestampType(), True),
            StructField("closedate", TimestampType(), True),
            StructField("closereasonid", IntegerType(), True),
            StructField("axisrequestid", StringType(), True),
            StructField("axissyncdate", TimestampType(), True),
            StructField("axispagecount", StringType(), True),
            StructField("isofflinepayment", StringType(), True),
            StructField("linkedrequests", StringType(), True),
            StructField("migrationreference", StringType(), True),
            StructField("originalldd", StringType(), True),
            StructField("identityverified", StringType(), True),
            StructField("ministrysignoffapproval", StringType(), True),
            StructField("requeststatuslabel", StringType(), True),
            StructField("isoipcreview", StringType(), True),
            StructField("recordspagecount", StringType(), True),
            StructField("axislanpagecount", StringType(), True),
            StructField("userrecordslockstatus", StringType(), True),
            StructField("estimatedpagecount", IntegerType(), True),
            StructField("estimatedtaggedpagecount", IntegerType(), True),
            StructField("isconsultflag", StringType(), True),
            StructField("isphasedrelease", StringType(), True)
        ]),
        "target_table_name": "foiministryrequests",
        "primary_keys": ["foiministryrequestid", "version"],
        "transform_map": {
            "foiministryrequestid": ("foiministryrequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "isactive": ("isactive", StringType()),
            "filenumber": ("filenumber", StringType()),
            "description": ("description", StringType()),
            "recordsearchfromdate": ("recordsearchfromdate", TimestampType()),
            "recordsearchtodate": ("recordsearchtodate", TimestampType()),
            "startdate": ("startdate", TimestampType()),
            "duedate": ("duedate", TimestampType()),
            "assignedto": ("assignedto", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "programareaid": ("programareaid", IntegerType()),
            "requeststatusid": ("requeststatusid", IntegerType()),
            "foirequest_id": ("foirequest_id", IntegerType()),
            "foirequestversion_id": ("foirequestversion_id", IntegerType()),
            "assignedgroup": ("assignedgroup", StringType()),
            "assignedministryperson": ("assignedministryperson", StringType()),
            "assignedministrygroup": ("assignedministrygroup", StringType()),
            "cfrduedate": ("cfrduedate", TimestampType()),
            "closedate": ("closedate", TimestampType()),
            "closereasonid": ("closereasonid", IntegerType()),
            "axisrequestid": ("axisrequestid", StringType()),
            "axissyncdate": ("axissyncdate", TimestampType()),
            "axispagecount": ("axispagecount", IntegerType()),
            "isofflinepayment": ("isofflinepayment", StringType()),
            "linkedrequests": ("linkedrequests", StringType()),
            "migrationreference": ("migrationreference", StringType()),
            "originalldd": ("originalldd", StringType()),
            "identityverified": ("identityverified", StringType()),
            "ministrysignoffapproval": ("ministrysignoffapproval", StringType()),
            "requeststatuslabel": ("requeststatuslabel", StringType()),
            "isoipcreview": ("isoipcreview", StringType()),
            "recordspagecount": ("recordspagecount", IntegerType()),
            "axislanpagecount": ("axislanpagecount", IntegerType()),
            "userrecordslockstatus": ("userrecordslockstatus", StringType()),
            "estimatedpagecount": ("estimatedpagecount", IntegerType()),
            "estimatedtaggedpagecount": ("estimatedtaggedpagecount", IntegerType()),
            "isconsultflag": ("isconsultflag", StringType()),
            "isphasedrelease": ("isphasedrelease", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiministryrequestsubjectcodes": {
        "source_table_schema": StructType([
            StructField("foiministrysubjectcodeid", IntegerType(), True),
            StructField("subjectcodeid", IntegerType(), True),
            StructField("foiministryrequestid", IntegerType(), True),
            StructField("foiministryrequestversion", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foiministryrequestsubjectcodes",
        "primary_keys": ["foiministrysubjectcodeid"],
        "transform_map": {
            "foiministrysubjectcodeid": ("foiministrysubjectcodeid", IntegerType()),
            "subjectcodeid": ("subjectcodeid", IntegerType()),
            "foiministryrequestid": ("foiministryrequestid", IntegerType()),
            "foiministryrequestversion": ("foiministryrequestversion", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequestcomments": {
        "source_table_schema": StructType([
            StructField("commentid", IntegerType(), True),
            StructField("parentcommentid", IntegerType(), True),
            StructField("requestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("comment", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("commenttypeid", IntegerType(), True),
            StructField("taggedusers", StringType(), True),
            StructField("commentsversion", IntegerType(), True)
        ]),
        "target_table_name": "foirawrequestcomments",
        "primary_keys": ["commentid", "commentsversion"],
        "transform_map": {
            "commentid": ("commentid", IntegerType()),
            "parentcommentid": ("parentcommentid", IntegerType()),
            "requestid": ("requestid", IntegerType()),
            "version": ("version", IntegerType()),
            "comment": ("comment", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isactive": ("isactive", StringType()),
            "commenttypeid": ("commenttypeid", IntegerType()),
            "taggedusers": ("taggedusers", StringType()),
            "commentsversion": ("commentsversion", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequestdocuments": {
        "source_table_schema": StructType([
            StructField("foidocumentid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("documentpath", StringType(), True),
            StructField("filename", StringType(), True),
            StructField("category", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foirequest_id", IntegerType(), True),
            StructField("foirequestversion_id", IntegerType(), True)
        ]),
        "target_table_name": "foirawrequestdocuments",
        "primary_keys": ["foidocumentid", "version"],
        "transform_map": {
            "foidocumentid": ("foidocumentid", IntegerType()),
            "version": ("version", IntegerType()),
            "documentpath": ("documentpath", StringType()),
            "filename": ("filename", StringType()),
            "category": ("category", StringType()),
            "isactive": ("isactive", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "foirequest_id": ("foirequest_id", IntegerType()),
            "foirequestversion_id": ("foirequestversion_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequestnotifications": {
        "source_table_schema": StructType([
            StructField("notificationid", IntegerType(), True),
            StructField("notification", StringType(), True),
            StructField("notificationtypeid", IntegerType(), True),
            StructField("requestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("idnumber", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("axisnumber", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("notificationtypelabel", StringType(), True)
        ]),
        "target_table_name": "foirawrequestnotifications",
        "primary_keys": ["notificationid"],
        "transform_map": {
            "notificationid": ("notificationid", IntegerType()),
            "notification": ("notification", StringType()),
            "notificationtypeid": ("notificationtypeid", IntegerType()),
            "requestid": ("requestid", IntegerType()),
            "version": ("version", IntegerType()),
            "idnumber": ("idnumber", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "axisnumber": ("axisnumber", StringType()),
            "isdeleted": ("isdeleted", StringType()),
            "notificationtypelabel": ("notificationtypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequestnotificationusers": {
        "source_table_schema": StructType([
            StructField("notificationuserid", IntegerType(), True),
            StructField("notificationid", IntegerType(), True),
            StructField("userid", StringType(), True),
            StructField("notificationusertypeid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("notificationusertypelabel", StringType(), True)
        ]),
        "target_table_name": "foirawrequestnotificationusers",
        "primary_keys": ["notificationuserid"],
        "transform_map": {
            "notificationuserid": ("notificationuserid", IntegerType()),
            "notificationid": ("notificationid", IntegerType()),
            "userid": ("userid", StringType()),
            "notificationusertypeid": ("notificationusertypeid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isdeleted": ("isdeleted", BooleanType()),
            "notificationusertypelabel": ("notificationusertypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequests": {
        "source_table_schema": StructType([
            StructField("requestid", IntegerType(), True),
            StructField("requestrawdata", StringType(), True),
            StructField("status", StringType(), True),
            StructField("notes", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("wfinstanceid", StringType(), True),
            StructField("version", IntegerType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("assignedto", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("sourceofsubmission", StringType(), True),
            StructField("assignedgroup", StringType(), True),
            StructField("ispiiredacted", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("requirespayment", StringType(), True),
            StructField("closedate", TimestampType(), True),
            StructField("closereasonid", IntegerType(), True),
            StructField("axisrequestid", StringType(), True),
            StructField("axissyncdate", TimestampType(), True),
            StructField("isiaorestricted", StringType(), True),
            StructField("linkedrequests", StringType(), True),
            StructField("requeststatuslabel", StringType(), True),
            StructField("isconsultflag", StringType(), True)
        ]),
        "target_table_name": "foirawrequests",
        "primary_keys": ["requestid", "version"],
        "transform_map": {
            "requestid": ("requestid", IntegerType()),
            "requestrawdata": ("requestrawdata", StringType()),
            "status": ("status", StringType()),
            "notes": ("notes", StringType()),
            "created_at": ("created_at", TimestampType()),
            "wfinstanceid": ("wfinstanceid", StringType()),
            "version": ("version", IntegerType()),
            "updated_at": ("updated_at", TimestampType()),
            "assignedto": ("assignedto", StringType()),
            "updatedby": ("updatedby", StringType()),
            "sourceofsubmission": ("sourceofsubmission", StringType()),
            "assignedgroup": ("assignedgroup", StringType()),
            "ispiiredacted": ("ispiiredacted", BooleanType()),
            "createdby": ("createdby", StringType()),
            "requirespayment": ("requirespayment", BooleanType()),
            "closedate": ("closedate", TimestampType()),
            "closereasonid": ("closereasonid", IntegerType()),
            "axisrequestid": ("axisrequestid", StringType()),
            "axissyncdate": ("axissyncdate", TimestampType()),
            "isiaorestricted": ("isiaorestricted", BooleanType()),
            "linkedrequests": ("linkedrequests", StringType()),
            "requeststatuslabel": ("requeststatuslabel", StringType()),
            "isconsultflag": ("isconsultflag", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirawrequestwatchers": {
        "source_table_schema": StructType([
            StructField("watcherid", IntegerType(), True),
            StructField("requestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("watchedbygroup", StringType(), True),
            StructField("watchedby", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "foirawrequestwatchers",
        "primary_keys": ["watcherid"],
        "transform_map": {
            "watcherid": ("watcherid", IntegerType()),
            "requestid": ("requestid", IntegerType()),
            "version": ("version", IntegerType()),
            "watchedbygroup": ("watchedbygroup", StringType()),
            "watchedby": ("watchedby", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestapplicantmappings": {
        "source_table_schema": StructType([
            StructField("foirequestapplicantmappingid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("requestortypeid", IntegerType(), True),
            StructField("foirequestapplicantid", IntegerType(), True),
            StructField("foirequest_id", IntegerType(), True),
            StructField("foirequestversion_id", IntegerType(), True),
            StructField("migrationreference", StringType(), True)
        ]),
        "target_table_name": "foirequestapplicantmappings",
        "primary_keys": ["foirequestapplicantmappingid"],
        "transform_map": {
            "foirequestapplicantmappingid": ("foirequestapplicantmappingid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "requestortypeid": ("requestortypeid", IntegerType()),
            "foirequestapplicantid": ("foirequestapplicantid", IntegerType()),
            "foirequest_id": ("foirequest_id", IntegerType()),
            "foirequestversion_id": ("foirequestversion_id", IntegerType()),
            "migrationreference": ("migrationreference", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestapplicants": {
        "source_table_schema": StructType([
            StructField("foirequestapplicantid", IntegerType(), True),
            StructField("firstname", StringType(), True),
            StructField("middlename", StringType(), True),
            StructField("lastname", StringType(), True),
            StructField("alsoknownas", StringType(), True),
            StructField("dob", StringType(), True),
            StructField("businessname", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("migrationreference", StringType(), True),
            StructField("applicantprofileid", StringType(), True),
            StructField("axisapplicantid", StringType(), True)
        ]),
        "target_table_name": "foirequestapplicants",
        "primary_keys": ["foirequestapplicantid"],
        "transform_map": {
            "foirequestapplicantid": ("foirequestapplicantid", IntegerType()),
            "firstname": ("firstname", StringType()),
            "middlename": ("middlename", StringType()),
            "lastname": ("lastname", StringType()),
            "alsoknownas": ("alsoknownas", StringType()),
            "dob": ("dob", DateType()),
            "businessname": ("businessname", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "migrationreference": ("migrationreference", StringType()),
            "applicantprofileid": ("applicantprofileid", StringType()),
            "axisapplicantid": ("axisapplicantid", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestapplicationfeereceipts": {
        "source_table_schema": StructType([
            StructField("receiptid", IntegerType(), True),
            StructField("applicationfeeid", IntegerType(), True),
            StructField("applicationfeeid_version", IntegerType(), True),
            StructField("receiptfilename", StringType(), True),
            StructField("receiptfilepath", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "foirequestapplicationfeereceipts",
        "primary_keys": ["receiptid"],
        "transform_map": {
            "receiptid": ("receiptid", IntegerType()),
            "applicationfeeid": ("applicationfeeid", IntegerType()),
            "applicationfeeid_version": ("applicationfeeid_version", IntegerType()),
            "receiptfilename": ("receiptfilename", StringType()),
            "receiptfilepath": ("receiptfilepath", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestapplicationfees": {
        "source_table_schema": StructType([
            StructField("applicationfeeid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("rawrequestid", IntegerType(), True),
            StructField("applicationfeestatus", StringType(), True),
            StructField("amountpaid", IntegerType(), True),
            StructField("paymentsource", StringType(), True),
            StructField("paymentdate", TimestampType(), True),
            StructField("orderid", StringType(), True),
            StructField("transactionnumber", StringType(), True),
            StructField("refundamount", StringType(), True),
            StructField("refunddate", StringType(), True),
            StructField("reasonforrefund", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foirequestapplicationfees",
        "primary_keys": ["applicationfeeid", "version"],
        "transform_map": {
            "applicationfeeid": ("applicationfeeid", IntegerType()),
            "version": ("version", IntegerType()),
            "rawrequestid": ("rawrequestid", IntegerType()),
            "applicationfeestatus": ("applicationfeestatus", StringType()),
            "amountpaid": ("amountpaid", IntegerType()),
            "paymentsource": ("paymentsource", StringType()),
            "paymentdate": ("paymentdate", TimestampType()),
            "orderid": ("orderid", StringType()),
            "transactionnumber": ("transactionnumber", StringType()),
            "refundamount": ("refundamount", DoubleType()),
            "refunddate": ("refunddate", TimestampType()),
            "reasonforrefund": ("reasonforrefund", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestcfrfees": {
        "source_table_schema": StructType([
            StructField("cfrfeeid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("ministryrequestversion", IntegerType(), True),
            StructField("feedata", StringType(), True),
            StructField("overallsuggestions", StringType(), True),
            StructField("cfrfeestatusid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("cfrformreasonid", IntegerType(), True)
        ]),
        "target_table_name": "foirequestcfrfees",
        "primary_keys": ["cfrfeeid", "version"],
        "transform_map": {
            "cfrfeeid": ("cfrfeeid", IntegerType()),
            "version": ("version", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "ministryrequestversion": ("ministryrequestversion", IntegerType()),
            "feedata": ("feedata", StringType()),
            "overallsuggestions": ("overallsuggestions", StringType()),
            "cfrfeestatusid": ("cfrfeestatusid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "cfrformreasonid": ("cfrformreasonid", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestcomments": {
        "source_table_schema": StructType([
            StructField("commentid", IntegerType(), True),
            StructField("parentcommentid", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("comment", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("commenttypeid", IntegerType(), True),
            StructField("taggedusers", StringType(), True),
            StructField("commentsversion", IntegerType(), True)
        ]),
        "target_table_name": "foirequestcomments",
        "primary_keys": ["commentid", "commentsversion"],
        "transform_map": {
            "commentid": ("commentid", IntegerType()),
            "parentcommentid": ("parentcommentid", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "comment": ("comment", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isactive": ("isactive", BooleanType()),
            "commenttypeid": ("commenttypeid", IntegerType()),
            "taggedusers": ("taggedusers", StringType()),
            "commentsversion": ("commentsversion", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestcontactinformation": {
        "source_table_schema": StructType([
            StructField("foirequestcontactid", IntegerType(), True),
            StructField("contactinformation", StringType(), True),
            StructField("dataformat", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("contacttypeid", IntegerType(), True),
            StructField("foirequest_id", IntegerType(), True),
            StructField("foirequestversion_id", IntegerType(), True),
            StructField("migrationreference", StringType(), True)
        ]),
        "target_table_name": "foirequestcontactinformation",
        "primary_keys": ["foirequestcontactid"],
        "transform_map": {
            "foirequestcontactid": ("foirequestcontactid", IntegerType()),
            "contactinformation": ("contactinformation", StringType()),
            "dataformat": ("dataformat", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "contacttypeid": ("contacttypeid", IntegerType()),
            "foirequest_id": ("foirequest_id", IntegerType()),
            "foirequestversion_id": ("foirequestversion_id", IntegerType()),
            "migrationreference": ("migrationreference", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestextensiondocumentmapping": {
        "source_table_schema": StructType([
            StructField("foirequestextensiondocumentid", IntegerType(), True),
            StructField("foirequestextensionid", IntegerType(), True),
            StructField("foiministrydocumentid", IntegerType(), True),
            StructField("extensionversion", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foirequestextensiondocumentmapping",
        "primary_keys": ["foirequestextensiondocumentid"],
        "transform_map": {
            "foirequestextensiondocumentid": ("foirequestextensiondocumentid", IntegerType()),
            "foirequestextensionid": ("foirequestextensionid", IntegerType()),
            "foiministrydocumentid": ("foiministrydocumentid", IntegerType()),
            "extensionversion": ("extensionversion", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestextensions": {
        "source_table_schema": StructType([
            StructField("foirequestextensionid", IntegerType(), True),
            StructField("extensionreasonid", IntegerType(), True),
            StructField("extensionstatusid", IntegerType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("extendedduedays", IntegerType(), True),
            StructField("extendedduedate", TimestampType(), True),
            StructField("decisiondate", StringType(), True),
            StructField("approvednoofdays", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("migrationreference", StringType(), True)
        ]),
        "target_table_name": "foirequestextensions",
        "primary_keys": ["foirequestextensionid", "version"],
        "transform_map": {
            "foirequestextensionid": ("foirequestextensionid", IntegerType()),
            "extensionreasonid": ("extensionreasonid", IntegerType()),
            "extensionstatusid": ("extensionstatusid", IntegerType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "extendedduedays": ("extendedduedays", IntegerType()),
            "extendedduedate": ("extendedduedate", TimestampType()),
            "decisiondate": ("decisiondate", TimestampType()),
            "approvednoofdays": ("approvednoofdays", IntegerType()),
            "version": ("version", IntegerType()),
            "isactive": ("isactive", BooleanType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "migrationreference": ("migrationreference", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestnotifications": {
        "source_table_schema": StructType([
            StructField("notificationid", IntegerType(), True),
            StructField("notification", StringType(), True),
            StructField("notificationtypeid", IntegerType(), True),
            StructField("requestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("idnumber", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("foirequestid", IntegerType(), True),
            StructField("axisnumber", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("notificationtypelabel", StringType(), True)
        ]),
        "target_table_name": "foirequestnotifications",
        "primary_keys": ["notificationid"],
        "transform_map": {
            "notificationid": ("notificationid", IntegerType()),
            "notification": ("notification", StringType()),
            "notificationtypeid": ("notificationtypeid", IntegerType()),
            "requestid": ("requestid", IntegerType()),
            "version": ("version", IntegerType()),
            "idnumber": ("idnumber", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "foirequestid": ("foirequestid", IntegerType()),
            "axisnumber": ("axisnumber", StringType()),
            "isdeleted": ("isdeleted", BooleanType()),
            "notificationtypelabel": ("notificationtypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestnotificationusers": {
        "source_table_schema": StructType([
            StructField("notificationuserid", IntegerType(), True),
            StructField("notificationid", IntegerType(), True),
            StructField("userid", StringType(), True),
            StructField("notificationusertypeid", IntegerType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isdeleted", StringType(), True),
            StructField("notificationusertypelabel", StringType(), True)
        ]),
        "target_table_name": "foirequestnotificationusers",
        "primary_keys": ["notificationuserid"],
        "transform_map": {
            "notificationuserid": ("notificationuserid", IntegerType()),
            "notificationid": ("notificationid", IntegerType()),
            "userid": ("userid", StringType()),
            "notificationusertypeid": ("notificationusertypeid", IntegerType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isdeleted": ("isdeleted", BooleanType()),
            "notificationusertypelabel": ("notificationusertypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestoipc": {
        "source_table_schema": StructType([
            StructField("oipcid", IntegerType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("oipcno", StringType(), True),
            StructField("investigator", StringType(), True),
            StructField("reviewtypeid", IntegerType(), True),
            StructField("reasonid", IntegerType(), True),
            StructField("statusid", IntegerType(), True),
            StructField("outcomeid", IntegerType(), True),
            StructField("isinquiry", StringType(), True),
            StructField("isjudicialreview", StringType(), True),
            StructField("issubsequentappeal", StringType(), True),
            StructField("inquiryattributes", StringType(), True),
            StructField("receiveddate", DateType(), True),
            StructField("closeddate", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foirequestoipc",
        "primary_keys": ["oipcid"],
        "transform_map": {
            "oipcid": ("oipcid", IntegerType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "oipcno": ("oipcno", StringType()),
            "investigator": ("investigator", StringType()),
            "reviewtypeid": ("reviewtypeid", IntegerType()),
            "reasonid": ("reasonid", IntegerType()),
            "statusid": ("statusid", IntegerType()),
            "outcomeid": ("outcomeid", IntegerType()),
            "isinquiry": ("isinquiry", BooleanType()),
            "isjudicialreview": ("isjudicialreview", BooleanType()),
            "issubsequentappeal": ("issubsequentappeal", BooleanType()),
            "inquiryattributes": ("inquiryattributes", StringType()),
            "receiveddate": ("receiveddate", DateType()),
            "closeddate": ("closeddate", TimestampType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestpayments": {
        "source_table_schema": StructType([
            StructField("paymentid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("foirequestid", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("ministryrequestversion", IntegerType(), True),
            StructField("paymenturl", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("paymentexpirydate", TimestampType(), True),
            StructField("paidamount", StringType(), True)
        ]),
        "target_table_name": "foirequestpayments",
        "primary_keys": ["paymentid", "version"],
        "transform_map": {
            "paymentid": ("paymentid", IntegerType()),
            "version": ("version", IntegerType()),
            "foirequestid": ("foirequestid", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "ministryrequestversion": ("ministryrequestversion", IntegerType()),
            "paymenturl": ("paymenturl", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "paymentexpirydate": ("paymentexpirydate", TimestampType()),
            "paidamount": ("paidamount", DoubleType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestpersonalattributes": {
        "source_table_schema": StructType([
            StructField("foirequestpersonalattributeid", IntegerType(), True),
            StructField("attributevalue", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("personalattributeid", IntegerType(), True),
            StructField("foirequest_id", IntegerType(), True),
            StructField("foirequestversion_id", IntegerType(), True)
        ]),
        "target_table_name": "foirequestpersonalattributes",
        "primary_keys": ["foirequestpersonalattributeid"],
        "transform_map": {
            "foirequestpersonalattributeid": ("foirequestpersonalattributeid", IntegerType()),
            "attributevalue": ("attributevalue", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "personalattributeid": ("personalattributeid", IntegerType()),
            "foirequest_id": ("foirequest_id", IntegerType()),
            "foirequestversion_id": ("foirequestversion_id", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestrecords": {
        "source_table_schema": StructType([
            StructField("recordid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("foirequestid", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("ministryrequestversion", IntegerType(), True),
            StructField("filename", StringType(), True),
            StructField("s3uripath", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("attributes", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("replacementof", StringType(), True)
        ]),
        "target_table_name": "foirequestrecords",
        "primary_keys": ["recordid", "version"],
        "transform_map": {
            "recordid": ("recordid", IntegerType()),
            "version": ("version", IntegerType()),
            "foirequestid": ("foirequestid", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "ministryrequestversion": ("ministryrequestversion", IntegerType()),
            "filename": ("filename", StringType()),
            "s3uripath": ("s3uripath", StringType()),
            "createdby": ("createdby", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "attributes": ("attributes", StringType()),
            "isactive": ("isactive", BooleanType()),
            "replacementof": ("replacementof", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequests": {
        "source_table_schema": StructType([
            StructField("foirequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("requesttype", StringType(), True),
            StructField("receiveddate", TimestampType(), True),
            StructField("isactive", StringType(), True),
            StructField("initialdescription", StringType(), True),
            StructField("initialrecordsearchfromdate", StringType(), True),
            StructField("initialrecordsearchtodate", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True),
            StructField("deliverymodeid", IntegerType(), True),
            StructField("receivedmodeid", IntegerType(), True),
            StructField("foirawrequestid", IntegerType(), True),
            StructField("applicantcategoryid", IntegerType(), True),
            StructField("wfinstanceid", StringType(), True),
            StructField("migrationreference", StringType(), True)
        ]),
        "target_table_name": "foirequests",
        "primary_keys": ["foirequestid", "version"],
        "transform_map": {
            "foirequestid": ("foirequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "requesttype": ("requesttype", StringType()),
            "receiveddate": ("receiveddate", TimestampType()),
            "isactive": ("isactive", BooleanType()),
            "initialdescription": ("initialdescription", StringType()),
            "initialrecordsearchfromdate": ("initialrecordsearchfromdate", TimestampType()),
            "initialrecordsearchtodate": ("initialrecordsearchtodate", TimestampType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "deliverymodeid": ("deliverymodeid", IntegerType()),
            "receivedmodeid": ("receivedmodeid", IntegerType()),
            "foirawrequestid": ("foirawrequestid", IntegerType()),
            "applicantcategoryid": ("applicantcategoryid", IntegerType()),
            "wfinstanceid": ("wfinstanceid", StringType()),
            "migrationreference": ("migrationreference", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequeststatuses": {
        "source_table_schema": StructType([
            StructField("requeststatusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("statuslabel", StringType(), True)
        ]),
        "target_table_name": "foirequeststatuses",
        "primary_keys": ["requeststatusid"],
        "transform_map": {
            "requeststatusid": ("requeststatusid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", BooleanType()),
            "statuslabel": ("statuslabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestteams": {
        "source_table_schema": StructType([
            StructField("requestteamid", IntegerType(), True),
            StructField("requesttype", StringType(), True),
            StructField("requeststatusid", IntegerType(), True),
            StructField("teamid", IntegerType(), True),
            StructField("programareaid", IntegerType(), True),
            StructField("isactive", StringType(), True),
            StructField("requeststatuslabel", StringType(), True)
        ]),
        "target_table_name": "foirequestteams",
        "primary_keys": ["requestteamid"],
        "transform_map": {
            "requestteamid": ("requestteamid", IntegerType()),
            "requesttype": ("requesttype", StringType()),
            "requeststatusid": ("requeststatusid", IntegerType()),
            "teamid": ("teamid", IntegerType()),
            "programareaid": ("programareaid", IntegerType()),
            "isactive": ("isactive", BooleanType()),
            "requeststatuslabel": ("requeststatuslabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirequestwatchers": {
        "source_table_schema": StructType([
            StructField("watcherid", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("watchedbygroup", StringType(), True),
            StructField("watchedby", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "foirequestwatchers",
        "primary_keys": ["watcherid"],
        "transform_map": {
            "watcherid": ("watcherid", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "watchedbygroup": ("watchedbygroup", StringType()),
            "watchedby": ("watchedby", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foirestrictedministryrequests": {
        "source_table_schema": StructType([
            StructField("restrictionid", IntegerType(), True),
            StructField("ministryrequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("type", StringType(), True),
            StructField("isrestricted", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "foirestrictedministryrequests",
        "primary_keys": ["restrictionid"],
        "transform_map": {
            "restrictionid": ("restrictionid", IntegerType()),
            "ministryrequestid": ("ministryrequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "type": ("type", StringType()),
            "isrestricted": ("isrestricted", BooleanType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiusers": {
        "source_table_schema": StructType([
            StructField("foiuserid", IntegerType(), True),
            StructField("username", StringType(), True),
            StructField("preferred_username", StringType(), True),
            StructField("firstname", StringType(), True),
            StructField("lastname", StringType(), True),
            StructField("email", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updatedby", StringType(), True)
        ]),
        "target_table_name": "foiusers",
        "primary_keys": ["foiuserid"],
        "transform_map": {
            "foiuserid": ("foiuserid", IntegerType()),
            "username": ("username", StringType()),
            "preferred_username": ("preferred_username", StringType()),
            "firstname": ("firstname", StringType()),
            "lastname": ("lastname", StringType()),
            "email": ("email", StringType()),
            "isactive": ("isactive", BooleanType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updatedby": ("updatedby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "historicalrecords": {
        "source_table_schema": StructType([
            StructField("historicalrecordid", IntegerType(), True),
            StructField("recordfilename", StringType(), True),
            StructField("description", StringType(), True),
            StructField("axisrequestid", StringType(), True),
            StructField("s3uripath", StringType(), True),
            StructField("createdby", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("attributes", StringType(), True),
            StructField("iscorresponcedocument", StringType(), True),
            StructField("displayfilename", StringType(), True)
        ]),
        "target_table_name": "historicalrecords",
        "primary_keys": ["historicalrecordid"],
        "transform_map": {
            "historicalrecordid": ("historicalrecordid", IntegerType()),
            "recordfilename": ("recordfilename", StringType()),
            "description": ("description", StringType()),
            "axisrequestid": ("axisrequestid", StringType()),
            "s3uripath": ("s3uripath", StringType()),
            "createdby": ("createdby", StringType()),
            "created_at": ("created_at", TimestampType()),
            "attributes": ("attributes", StringType()),
            "iscorresponcedocument": ("iscorresponcedocument", BooleanType()),
            "displayfilename": ("displayfilename", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "notificationtypes": {
        "source_table_schema": StructType([
            StructField("notificationtypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("notificationtypelabel", StringType(), True)
        ]),
        "target_table_name": "notificationtypes",
        "primary_keys": ["notificationtypeid"],
        "transform_map": {
            "notificationtypeid": ("notificationtypeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", BooleanType()),
            "notificationtypelabel": ("notificationtypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "notificationusertypes": {
        "source_table_schema": StructType([
            StructField("notificationusertypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("notificationusertypelabel", StringType(), True)
        ]),
        "target_table_name": "notificationusertypes",
        "primary_keys": ["notificationusertypeid"],
        "transform_map": {
            "notificationusertypeid": ("notificationusertypeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", BooleanType()),
            "notificationusertypelabel": ("notificationusertypelabel", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcinquiryoutcomes": {
        "source_table_schema": StructType([
            StructField("inquiryoutcomeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcinquiryoutcomes",
        "primary_keys": ["inquiryoutcomeid"],
        "transform_map": {
            "inquiryoutcomeid": ("inquiryoutcomeid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcoutcomes": {
        "source_table_schema": StructType([
            StructField("outcomeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcoutcomes",
        "primary_keys": ["outcomeid"],
        "transform_map": {
            "outcomeid": ("outcomeid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcreasons": {
        "source_table_schema": StructType([
            StructField("reasonid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcreasons",
        "primary_keys": ["reasonid"],
        "transform_map": {
            "reasonid": ("reasonid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcreviewtypes": {
        "source_table_schema": StructType([
            StructField("reviewtypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcreviewtypes",
        "primary_keys": ["reviewtypeid"],
        "transform_map": {
            "reviewtypeid": ("reviewtypeid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcreviewtypesreasons": {
        "source_table_schema": StructType([
            StructField("reviewtypereasonid", IntegerType(), True),
            StructField("reviewtypeid", IntegerType(), True),
            StructField("reasonid", IntegerType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcreviewtypesreasons",
        "primary_keys": ["reviewtypereasonid"],
        "transform_map": {
            "reviewtypereasonid": ("reviewtypereasonid", IntegerType()),
            "reviewtypeid": ("reviewtypeid", IntegerType()),
            "reasonid": ("reasonid", IntegerType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "oipcstatuses": {
        "source_table_schema": StructType([
            StructField("statusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "oipcstatuses",
        "primary_keys": ["statusid"],
        "transform_map": {
            "statusid": ("statusid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "operatingteamemails": {
        "source_table_schema": StructType([
            StructField("emailid", IntegerType(), True),
            StructField("teamid", IntegerType(), True),
            StructField("email_address", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("updated_at", TimestampType(), True)
        ]),
        "target_table_name": "operatingteamemails",
        "primary_keys": ["emailid"],
        "transform_map": {
            "emailid": ("emailid", IntegerType()),
            "teamid": ("teamid", IntegerType()),
            "email_address": ("email_address", StringType()),
            "isactive": ("isactive", StringType()),
            "created_at": ("created_at", TimestampType()),
            "updated_at": ("updated_at", TimestampType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "operatingteams": {
        "source_table_schema": StructType([
            StructField("teamid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("type", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "operatingteams",
        "primary_keys": ["teamid"],
        "transform_map": {
            "teamid": ("teamid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "type": ("type", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "payments": {
        "source_table_schema": StructType([
            StructField("payment_id", IntegerType(), True),
            StructField("fee_code_id", IntegerType(), True),
            StructField("quantity", IntegerType(), True),
            StructField("total", DoubleType(), True),
            StructField("status", StringType(), True),
            StructField("request_id", IntegerType(), True),
            StructField("created_on", TimestampType(), True),
            StructField("completed_on", StringType(), True),
            StructField("paybc_url", StringType(), True),
            StructField("response_url", StringType(), True),
            StructField("order_id", StringType(), True),
            StructField("transaction_number", StringType(), True)
        ]),
        "target_table_name": "payments",
        "primary_keys": ["payment_id"],
        "transform_map": {
            "payment_id": ("payment_id", IntegerType()),
            "fee_code_id": ("fee_code_id", IntegerType()),
            "quantity": ("quantity", IntegerType()),
            "total": ("total", DoubleType()),
            "status": ("status", StringType()),
            "request_id": ("request_id", IntegerType()),
            "created_on": ("created_on", TimestampType()),
            "completed_on": ("completed_on", StringType()),
            "paybc_url": ("paybc_url", StringType()),
            "response_url": ("response_url", StringType()),
            "order_id": ("order_id", StringType()),
            "transaction_number": ("transaction_number", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "personalinformationattributes": {
        "source_table_schema": StructType([
            StructField("attributeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "personalinformationattributes",
        "primary_keys": ["attributeid"],
        "transform_map": {
            "attributeid": ("attributeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "programareadivisions": {
        "source_table_schema": StructType([
            StructField("divisionid", IntegerType(), True),
            StructField("programareaid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("sortorder", IntegerType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("issection", StringType(), True),
            StructField("parentid", IntegerType(), True),
            StructField("specifictopersonalrequests", StringType(), True),
            StructField("type", StringType(), True)
        ]),
        "target_table_name": "programareadivisions",
        "primary_keys": ["divisionid"],
        "transform_map": {
            "divisionid": ("divisionid", IntegerType()),
            "programareaid": ("programareaid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "sortorder": ("sortorder", IntegerType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "issection": ("issection", StringType()),
            "parentid": ("parentid", IntegerType()),
            "specifictopersonalrequests": ("specifictopersonalrequests", StringType()),
            "type": ("type", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "programareadivisionstages": {
        "source_table_schema": StructType([
            StructField("stageid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True)
        ]),
        "target_table_name": "programareadivisionstages",
        "primary_keys": ["stageid"],
        "transform_map": {
            "stageid": ("stageid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", StringType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "programareas": {
        "source_table_schema": StructType([
            StructField("programareaid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("type", StringType(), True),
            StructField("isactive", StringType(), True),
            StructField("bcgovcode", StringType(), True),
            StructField("iaocode", StringType(), True)
        ]),
        "target_table_name": "programareas",
        "primary_keys": ["programareaid"],
        "transform_map": {
            "programareaid": ("programareaid", IntegerType()),
            "name": ("name", StringType()),
            "type": ("type", StringType()),
            "isactive": ("isactive", StringType()),
            "bcgovcode": ("bcgovcode", StringType()),
            "iaocode": ("iaocode", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "receivedmodes": {
        "source_table_schema": StructType([
            StructField("receivedmodeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "receivedmodes",
        "primary_keys": ["receivedmodeid"],
        "transform_map": {
            "receivedmodeid": ("receivedmodeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "requestortypes": {
        "source_table_schema": StructType([
            StructField("requestortypeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "requestortypes",
        "primary_keys": ["requestortypeid"],
        "transform_map": {
            "requestortypeid": ("requestortypeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "revenueaccounts": {
        "source_table_schema": StructType([
            StructField("revenue_account_id", IntegerType(), True),
            StructField("client", IntegerType(), True),
            StructField("responsibility_centre", IntegerType(), True),
            StructField("service_line", IntegerType(), True),
            StructField("stob", IntegerType(), True),
            StructField("project_code", IntegerType(), True)
        ]),
        "target_table_name": "revenueaccounts",
        "primary_keys": ["revenue_account_id"],
        "transform_map": {
            "revenue_account_id": ("revenue_account_id", IntegerType()),
            "client": ("client", IntegerType()),
            "responsibility_centre": ("responsibility_centre", IntegerType()),
            "service_line": ("service_line", IntegerType()),
            "stob": ("stob", IntegerType()),
            "project_code": ("project_code", IntegerType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "subjectcodes": {
        "source_table_schema": StructType([
            StructField("subjectcodeid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("description", StringType(), True),
            StructField("isaxissubjectcode", StringType(), True),
            StructField("isactive", StringType(), True)
        ]),
        "target_table_name": "subjectcodes",
        "primary_keys": ["subjectcodeid"],
        "transform_map": {
            "subjectcodeid": ("subjectcodeid", IntegerType()),
            "name": ("name", StringType()),
            "description": ("description", StringType()),
            "isaxissubjectcode": ("isaxissubjectcode", StringType()),
            "isactive": ("isactive", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "unopenedreport": {
        "source_table_schema": StructType([
            StructField("id", IntegerType(), True),
            StructField("rawrequestid", IntegerType(), True),
            StructField("date", TimestampType(), True),
            StructField("rank", IntegerType(), True),
            StructField("potentialmatches", StringType(), True)
        ]),
        "target_table_name": "unopenedreport",
        "primary_keys": ["id"],
        "transform_map": {
            "id": ("id", IntegerType()),
            "rawrequestid": ("rawrequestid", IntegerType()),
            "date": ("date", TimestampType()),
            "rank": ("rank", IntegerType()),
            "potentialmatches": ("potentialmatches", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "foiopeninformationrequests": {
        "source_table_schema": StructType([
            StructField("foiopeninforequestid", IntegerType(), True),
            StructField("version", IntegerType(), True),
            StructField("foiministryrequest_id", IntegerType(), True),
            StructField("foiministryrequestversion_id", IntegerType(), True),
            StructField("oiexemption_id", IntegerType(), True),
            StructField("oipublicationstatus_id", IntegerType(), True),
            StructField("oiassignedto", StringType(), True),
            StructField("pagereference", StringType(), True),
            StructField("oiexemptionapproved", BooleanType(), True),
            StructField("iaorationale", StringType(), True),
            StructField("oifeedback", StringType(), True),
            StructField("publicationdate", TimestampType(), True),
            StructField("receiveddate", TimestampType(), True),
            StructField("copyrightsevered", BooleanType(), True),
            StructField("isactive", BooleanType(), True),
            StructField("created_at", TimestampType(), True),
            StructField("createdby", StringType(), True),
            StructField("updated_at", TimestampType(), True),
            StructField("updatedby", StringType(), True),
            StructField("processingstatus", StringType(), True),
            StructField("processingmessage", StringType(), True),
            StructField("sitemap_pages", StringType(), True)
        ]),
        "target_table_name": "foiopeninformationrequests",
        "primary_keys": ["foiopeninforequestid"],
        "transform_map": {
            "foiopeninforequestid": ("foiopeninforequestid", IntegerType()),
            "version": ("version", IntegerType()),
            "foiministryrequest_id": ("foiministryrequest_id", IntegerType()),
            "foiministryrequestversion_id": ("foiministryrequestversion_id", IntegerType()),
            "oiexemption_id": ("oiexemption_id", IntegerType()),
            "oipublicationstatus_id": ("oipublicationstatus_id", IntegerType()),
            "oiassignedto": ("oiassignedto", StringType()),
            "pagereference": ("pagereference", StringType()),
            "oiexemptionapproved": ("oiexemptionapproved", BooleanType()),
            "iaorationale": ("iaorationale", StringType()),
            "oifeedback": ("oifeedback", StringType()),
            "publicationdate": ("publicationdate", TimestampType()),
            "receiveddate": ("receiveddate", TimestampType()),
            "copyrightsevered": ("copyrightsevered", BooleanType()),
            "isactive": ("isactive", BooleanType()),
            "created_at": ("created_at", TimestampType()),
            "createdby": ("createdby", StringType()),
            "updated_at": ("updated_at", TimestampType()),
            "updatedby": ("updatedby", StringType()),
            "processingstatus": ("processingstatus", StringType()),
            "processingmessage": ("processingmessage", StringType()),
            "sitemap_pages": ("sitemap_pages", StringType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "openinfopublicationstatuses": {
        "source_table_schema": StructType([
            StructField("oipublicationstatusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", BooleanType(), True)
        ]),
        "target_table_name": "openinfopublicationstatuses",
        "primary_keys": ["oipublicationstatusid"],
        "transform_map": {
            "oipublicationstatusid": ("oipublicationstatusid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "openinformationexemptions": {
        "source_table_schema": StructType([
            StructField("oiexemptionid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", BooleanType(), True)
        ]),
        "target_table_name": "openinformationexemptions",
        "primary_keys": ["oiexemptionid"],
        "transform_map": {
            "oiexemptionid": ("oiexemptionid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    },
    "openinformationstatuses": {
        "source_table_schema": StructType([
            StructField("oistatusid", IntegerType(), True),
            StructField("name", StringType(), True),
            StructField("isactive", BooleanType(), True)
        ]),
        "target_table_name": "openinformationstatuses",
        "primary_keys": ["oistatusid"],
        "transform_map": {
            "oistatusid": ("oistatusid", IntegerType()),
            "name": ("name", StringType()),
            "isactive": ("isactive", BooleanType()),
            "load_timestamp": (None, TimestampType(), lambda: current_timestamp())
        }
    }
}

In [0]:
# Helper Functions

def get_table_config_from_path(file_path, mappings):
    for key, config in mappings.items():
        if (f"{key}.json") in file_path.lower():
            return config
    raise ValueError(f"No table mapping found for file path: {file_path}")

def get_dynamic_select_expressions(source_df, target_schema_actual: StructType, transform_map):
    select_exprs = []
    source_schema_dict = {f.name.lower(): f.dataType for f in source_df.schema.fields}

    for target_field in target_schema_actual.fields:
        target_col_name = target_field.name
        target_type = target_field.dataType
        
        transform_rule = transform_map.get(target_col_name)

        if transform_rule:
            src_col_name_or_none, rule_target_type, *custom_transform_func = transform_rule
            
            if custom_transform_func:
                select_exprs.append(custom_transform_func[0]().alias(target_col_name))
            elif src_col_name_or_none:
                if "." in src_col_name_or_none:
                    select_exprs.append(col(src_col_name_or_none).cast(target_type).alias(target_col_name))
                elif src_col_name_or_none.lower() in source_schema_dict:
                    select_exprs.append(col(src_col_name_or_none).cast(target_type).alias(target_col_name))
                else:
                    print(f"Warning: Source column '{src_col_name_or_none}' not found for target column '{target_col_name}'. Adding as NULL.")
                    select_exprs.append(lit(None).cast(target_type).alias(target_col_name))
            else:
                print(f"Warning: No source column specified for target column '{target_col_name}' and no custom transform. Adding as NULL.")
                select_exprs.append(lit(None).cast(target_type).alias(target_col_name))
        else:
            if target_col_name.lower() in source_schema_dict:
                select_exprs.append(col(target_col_name).cast(target_type).alias(target_col_name))
                print(f"Info: Target column '{target_col_name}' in actual schema found in source but not in transform_map. Selecting directly.")
            else:
                select_exprs.append(lit(None).cast(target_type).alias(target_col_name))
                print(f"Info: Target column '{target_col_name}' in actual schema not found in source or transform_map. Adding as NULL.")
            
    return select_exprs


def create_or_get_delta_table(spark_session, base_path, table_name):
    """
    Retrieves the DeltaTable object and its schema if it exists.
    If the table does not exist, it returns (None, None).
    """
    full_table_path = f"{base_path}/{table_name}"
    
    if not DeltaTable.isDeltaTable(spark_session, full_table_path):
        print(f"  Delta table '{table_name}' does NOT exist at {full_table_path}. Skipping this file.")
        return None, None # Signal that the table was not found
    else:
        print(f"  Delta table '{table_name}' already exists at {full_table_path}. Retrieving schema.")
        actual_table_schema = spark_session.read.format("delta").load(full_table_path).schema
        print(f"  Retrieved actual schema for {table_name}:")
        spark_session.createDataFrame([], actual_table_schema).printSchema()
        delta_table_ref = DeltaTable.forPath(spark_session, full_table_path)
        return delta_table_ref, actual_table_schema

In [0]:
try:
    last_successful_run_time = get_last_successful_run_time(spark, JOB_TRACKER_TABLE_PATH, JOB_NAME)
    # date_range = generate_date_range_json(last_successful_run_time, job_start_timestamp)
    date_range = generate_date_range_json(datetime.strptime('2025-08-25', '%Y-%m-%d'), datetime.strptime('2025-08-25', '%Y-%m-%d'))
    # print(f"Type of date_range ({date_range}): {type(date_range)}")

    # Enable automatic schema evolution for merge operations
    spark.conf.set("spark.databricks.delta.schema.autoMerge.enabled", "true")

    # Main Processing Loop
    print(f"Starting JSON to Delta merge process... Date range: {date_range}")

    for date_str in date_range:
        print(f"Processing data for date: {date_str}")

        for json_file_relative_path in json_files_to_process:
            full_json_path = f"{S3_BASE_PATH}{date_str}/{json_file_relative_path}"
            print(f"\n--- Processing file: {full_json_path} ---")

            try:
                # 1. get target table configuration
                table_config = get_table_config_from_path(json_file_relative_path, table_mappings)
                target_table_name = table_config["target_table_name"]
                primary_keys = table_config["primary_keys"]
                transform_map = table_config["transform_map"]
                
                full_delta_table_path = f"{DELTA_BASE_PATH}/{target_table_name}"

                print(f"  Target table: {target_table_name} at {full_delta_table_path}")
                print(f"  Primary Keys: {primary_keys}")

                # 2. Get the Delta table reference and schema
                delta_table, actual_target_schema = create_or_get_delta_table(
                    spark, DELTA_BASE_PATH, target_table_name # No initial_schema_for_creation passed here
                )
                
                # --- Skip if table does not exist ---
                if delta_table is None:
                    continue # Skip to the next file in the loop

                # 3. Read the JSON file
                try:
                    source_df = spark.read.schema(table_config["source_table_schema"]).json(full_json_path)
                    print("  Source JSON schema (inferred):")
                    source_df.printSchema()
                except Exception as e:
                    # Skip to next file/table if JSON file not exist
                    if "Path does not exist" in str(e) or "No such file or directory" in str(e):
                        print(f"S3 JSON file '{full_json_path}' not found. Skipping this file.")
                        continue # Skip to the next file in the loop
                    else:
                        print(f"Other S3 JSON file '{full_json_path}' error. Escalate issue. {e}")
                        raise e

                # 4. Transform source data to match target schema
                print("start transform:")
                select_expressions = get_dynamic_select_expressions(source_df, actual_target_schema, transform_map)
                
                transformed_df = source_df.select(*select_expressions)
                print("  Transformed DataFrame schema (before merge):")
                transformed_df.printSchema()
                
                # 5. UPSERT/MERGE
                print("start merge:")
                merge_condition = " AND ".join([f"target.{pk} = source.{pk}" for pk in primary_keys])

                print(f"  Merge condition: {merge_condition}")
                
                (delta_table.alias("target")
                    .merge(
                        transformed_df.alias("source"),
                        merge_condition
                    )
                    .whenMatchedUpdateAll()
                    .whenNotMatchedInsertAll()
                    .execute()
                )
                print(f"  Successfully merged data from {json_file_relative_path} into {target_table_name}.")

            except ValueError as ve:
                print(f"Error processing {json_file_relative_path}: {ve}")
                # raise ve
            except Exception as e:
                print(f"An unexpected error occurred for {json_file_relative_path}: {e}")
                raise e
            
        print(f"Processing date: {date_str} - completed")
    
    print("\nJSON to Delta merge process completed.")
    job_status = "SUCCEEDED"
    status_message = f"Job completed successfully. Dates processed: {date_range}" # Optionally add to message

except Exception as e:
    job_status = "FAILED"
    status_message = f"Job failed: {e}"
    print(f"ERROR: {status_message}")
    # sys.exit(1) # Exit with a non-zero code to indicate failure in Databricks Jobs

finally:
    # 3. After the job, save job status (add end time and update the status)
    job_end_timestamp = datetime.now()
    
    record_job_status(
        spark=spark,
        job_tracker_table_path=JOB_TRACKER_TABLE_PATH,
        job_name=JOB_NAME,
        run_id=current_run_id,
        status=job_status,
        start_time=job_start_timestamp,
        end_time=job_end_timestamp,
        message=status_message # Include the message with date range if desired
    )
    
    # spark.stop()
    print(f"Job '{JOB_NAME}' (run_id: {current_run_id}) finished with status: {job_status}")

In [0]:
# %sql
# select * from foi_mod.foirawrequests
# order by requestid desc
# limit 200;
